In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
import seaborn as sns
import re
import requests

%matplotlib inline

In [21]:
papers = pd.read_csv(os.getcwd().replace('scripts/old', 'data/papers') + '/final_valid_papers.csv')

In [22]:
kwords = pd.read_csv(os.getcwd().replace('scripts/old', 'data')+'/technical_buzzwords.csv')
words = pd.DataFrame(columns = ['topic', 'keywords'])
for i in range(kwords.shape[0]):
    words.loc[len(words.index)] = [kwords.iat[i,0], kwords.iat[i,1].split(", ")]



In [23]:
papers

journal_id  volume_id  year  \
0               1          0  2023   
1               1          0  2023   
2               1          0  2023   
3               1          0  2023   
4               1          0  2023   
...           ...        ...   ...   
26469          28         68  2018   
26470          28         68  2018   
26471          28         68  2018   
26472          28         68  2018   
26473          28         68  2018   

                                                    link  \
0      https://www.sciencedirect.com/science/article/...   
1      https://www.sciencedirect.com/science/article/...   
2      https://www.sciencedirect.com/science/article/...   
3      https://www.sciencedirect.com/science/article/...   
4      https://www.sciencedirect.com/science/article/...   
...                                                  ...   
26469  https://www.sciencedirect.com/science/article/...   
26470  https://www.sciencedirect.com/science/article/...   
26471  https://www.sciencedirect.com/science/article/...   
26472  https://www.sciencedirect.com/science/article/...   
26473  https://www.sciencedirect.com/science/article/...   

                                   doi        date                pii  \
0      10.1016/j.advwatres.2023.104474  2023-08-01  S0309170823001094   
1      10.1016/j.advwatres.2023.104475  2023-08-01  S0309170823001100   
2      10.1016/j.advwatres.2023.104476  2023-08-01  S0309170823001112   
3      10.1016/j.advwatres.2023.104477  2023-08-01  S0309170823001124   
4      10.1016/j.advwatres.2023.104485  2023-08-01  S0309170823001203   
...                                ...         ...                ...   
26469    10.1016/j.envsoft.2017.09.020  2018-01-01  S1364815217304140   
26470    10.1016/j.envsoft.2017.09.010  2018-01-01  S1364815216304005   
26471    10.1016/j.envsoft.2017.08.002  2018-01-01  S1364815216309550   
26472    10.1016/j.envsoft.2017.09.011  2018-01-01  S136481521630843X   
26473    10.1016/j.envsoft.2017.09.002  2018-01-01  S136481521630648X   

                      eid     scopus_id  citations  \
0      2-s2.0-85161258968  8.516126e+10        0.0   
1      2-s2.0-85161636300  8.516164e+10        0.0   
2      2-s2.0-85161667219  8.516167e+10        0.0   
3      2-s2.0-85161661769  8.516166e+10        0.0   
4      2-s2.0-85161678744  8.516168e+10        0.0   
...                   ...           ...        ...   
26469  2-s2.0-85032268729  8.503227e+10      142.0   
26470  2-s2.0-85032353211  8.503235e+10       54.0   
26471  2-s2.0-85032684995  8.503268e+10        5.0   
26472  2-s2.0-85032736133  8.503274e+10       45.0   
26473  2-s2.0-85034039255  8.503404e+10       34.0   

                                                 authors  \
0                  58307014400, 36008901600, 26635952600   
1                               53984251100, 35610240400   
2                                            25822520800   
3                   57210424267, 36160282300, 7404296426   
4                                56708113100, 7202660185   
...                                                  ...   
26469  34571084000, 55188785600, 33767605600, 5721638...   
26470  37111876100, 15069126500, 57189996027, 3574567...   
26471  7006615627, 49661600900, 15132591800, 66033150...   
26472  56111768300, 37010006200, 43361846100, 5611302...   
26473  52365181700, 22957372900, 7005864023, 89834056...   

                                            affiliations  open_access  
0                             60031268:60031268:60007989         True  
1                            60104134:60104134, 60022461         True  
2                                               60025063         True  
3                             60018394:60018394:60018394         True  
4                                      60030447:60007566         True  
...                                                  ...          ...  
26469  60016437:60012197:60012197:60012197, 60025278:...         True  
26470 

In [28]:
keys = [
    "17ca4a831e62a9479b79d3185e2bb7c9",
    "8136f24479fb8bd24432478e10d5d69c",
    "f907ed21538230d82be259a24df59042",
    "e7fb9860807a25abf5ae0b287f32a5cc",
    "78f68de19035f872bcc55fd70705b427",
    "7baf5abc97d978fee2bc4120305aedcb",
    "b31320020fca70a856a402ee7e02ec9b",
    "d39d7f9cb431a873853066d351bc2c4f",
    "0ec3c0ffd88e2da0a35b43d8248b8e21"
]

In [26]:
def find_keywords(k_list, abstract):
    ret = []
    
    old_abstract = abstract
    
    orig = old_abstract
    
    for word in k_list:
        word = word.lower().replace('-', ' ')
        curr_abstract = old_abstract
        c_orig = orig
        done = False
        while(True):
            if done:
                break
            try:
                found_i = curr_abstract.index(word)

                l_b = max(0, found_i - 100)
                u_b = min(len(curr_abstract), found_i+len(word)+100)
                
                if(l_b > u_b):
                    break
                try:
                    ret.append([word,c_orig[l_b:u_b]])
                except TypeError:
                    print(c_orig)
                    print(l_b + " " + u_b)
                
#                 print(word + " "+ curr_abstract[l_b:u_b])
                
                if(u_b == len(curr_abstract)):
                    break
                
                curr_abstract = curr_abstract[found_i + len(word):]
                c_orig = c_orig[found_i + len(word):]

            except ValueError:
                done = True
                continue
    
    
    return ret#returns the specific word & context

In [10]:
find_keywords(words.iat[0, 1], "In this paper, we report on experimental and computational studies investigating the evolution of pore structure and permeability of a microporous carbonate rock during chemical dissolution using information provided from X-ray micro-computed tomography (µ-CT) and mercury intrusion porosimetry (MIP) techniques. We consider a chemical dissolution in a core sample by a nonacidic solution wherein a quasi-uniform modification of pore structure occurred. First, we conduct a comparative analysis on the capabilities and limitations of the µ-CT and MIP methods to quantify the evolution of pore size and pore surface area. In particular, we incorporate micropores into the calculations of the pore size-related parameters to highlight the uncertainties that ignoring microporosity causes. In the second part of the paper, we present predictive results on the fractional changes in the permeability of the rock using the Katz-Thompson (KT) and Kozeny-Carman (KC) models. The predictions are based on two characteristic parameters of the pore phase, i.e., the critical pore diameter and the specific pore surface area, and two characteristic macro-scale properties, i.e., porosity and formation factor, all calculated from both the MIP and µ-CT methods. The predicted changes in permeability are compared with those calculated directly on the images using the lattice-Boltzmann (LB) flow simulation on segmented images. The effect of changes in microporosity is also assessed to investigate its role in permeability evolution. The results showed that the KT model could reasonably predict the fractional changes in permeability in terms of the pore structure and macroscale parameters calculated from the MIP and µ-CT methods. Keywords Carbonate dissolution Pore-scale µ-CT imaging Mercury porosimetry Pore surface area Permeability Microporosity Data Availability Data will be made available on request. Nomenclature Ai Incremental specific pore area for the ith step of mercury injection Ait Cumulative (total) specific pore surface area d Diameter of the mercury intruded cylindrical pore Dave Average pore diameter Dc Critical pore diameter, determined from mercury intrusion curves Dc,mac Macro critical pore diameter Dc,mic Micro critical pore diameter Dm Mean pore diameter Dmed,A Median pore-throat diameter (by area) Dmed,V Median pore-throat diameter (by volume) Dth Threshold pore-throat diameter F Electrical formation factor of the rock Ii Cumulative specific intrusion volume for the ith step of mercury injection Iit Cumulative specific intrusion volume at the end of MIP test Ji Incremental specific intrusion volume for the ith step of mercury injection Jit Total incremental specific intrusion volume J i * Percentage of the intrusion mercury volume at the ith step of injection k Permeability lc Critical pore diameter, used in Katz-Thompson model l m a x e Pore size at which hydraulic conductance is maximum Npx The number of pore-solid face pixels P Applied mercury pressure R Relative change in pore-throat number Spx Pixel surface area Sμ-CT Specific pore surface area from μ-CT data SMIP,t Cumulative (total) specific pore surface area estimated from MIP SMIP,mac Specific surface area of macropores estimated from MIP SP Specific pore surface area S ( l m a x e ) Fraction of total porosity filled by mercury at l m a x e Vi Intruded volume at the ith step of mercury injection Vimg Image volume Greek symbols γ Surface tension θ Contact angle ϕ Porosity 1 Introduction The transport of fluids accompanying chemical reactions in sedimentary rocks presents a complex problem that is central in a wide variety of geochemical processes such as acidizing of petroleum reservoirs (Ali and Nasr-El-Din, 2020), contaminant transport in groundwater resources (Nick et al., 2013), and geological storage of CO2 (Liu et al., 2019) and hydrogen (Hassanpouryouzband et al., 2021). These processes often induce substantial changes in rock microstructure which, in turn, may significantly change the petrophysical and transport properties of the rock. In the petroleum industry, billions of dollars are spent annually on oil and gas exploration and developing complex and risky reservoirs, particularly carbonate reservoirs. During the life cycle of a reservoir, chemical dissolution plays an important role, from treating formation damage resulting from drilling operations to using acids and other chemicals for improved and enhanced oil recovery. A better understanding of chemical dissolution at the pore scale can, for example, lead to improvement in the performance of reactive processes during the development of reservoirs and hence dramatically reduce the associated costs. Changes in petrophysical and transport properties of sedimentary rocks strongly depend on the evolution of pore structure including pore/throat shape and size, pore surface area, and connectivity of the pore system (Garing et al., 2014; Qajar and Arns, 2017). Among sedimentary rocks, carbonates show particular features and are much more susceptible to alteration than sandstones when exposed to the flow of reactive fluids in subsurface formations. The characterization of changes in the microstructure of carbonate rocks is extremely challenging because these rocks often exhibit complex microstructure and disorder over a wide range of length scales (Knackstedt et al., 2007). This complexity causes considerable variability in relations of structure and topology to physical properties. Most carbonate rocks comprise a variety of pore and grain types with irregular spatial distributions and a wide spectrum of pore sizes from nanometers to centimeters and hence the macro-scale properties of carbonate rocks, e.g., permeability and electrical conductivity, are highly variable (Ramakrishnan et al., 2001; Arns et al., 2004). As a result, it has been of great importance to accurately characterize the pore structure of carbonate rocks, its evolution during geochemical processes, and the relation between the evolution in pore-scale descriptors and changes in macroscale properties. Two widely techniques used to characterize the pore structure of porous materials, including conventional and unconventional reservoir rocks, are mercury intrusion porosimetry (MIP) and X-ray computed tomography (CT)—especially microtomography (μ-CT). The MIP method has primarily been used for decades to determine the pore size distribution (PSD) and pore volume of various porous media (Schmitt et al., 2015; Sidiq et al., 2020). This method is based on a bundle of capillary tubes model, contrary to a real porous structure wherein a complex network of pores and throats exists. Hence some authors have referred to the MIP-based void size distribution as pore access size distribution (Feia et al., 2014). Nonetheless, the high-pressure MIP makes it possible to probe the nano-scale void space of porous materials, particularly tight rocks (Ma et al., 2020). From MIP-based pore-throat size distribution, it is possible to calculate a range of pore- and core-scale properties (Purcell, 1949; Liu and Winslow, 1995; Aligizaki, 2005). In recent years, in particular, the use of X-ray μ-CT and the image-based simulations, known as digital rock physics (DRP), is a rapidly evolving research field and is increasingly being used to quantitatively characterize the microstructure and properties of materials via tomographically recorded 3D media (Flannery et al., 1987; Spanne et al., 1994; Arns et al., 2003; Derzhi et al., 2010; Knackstedt et al., 2011; Andrä et al., 2013; Cnudde and Boone, 2013; Berg et al., 2017; Hassanpouryouzband et al., 2020; Wang and Miller, 2020; Ni et al., 2021). In the μ-CT technique, high-resolution images of the material specimen are acquired at micro- and nano-meter scales and processed to create accurate 3D digital data. The material structure parameters and physical properties are then computed using advanced numerical techniques at the pore scale. The main advantage of the μ-CT method is the ability to nondestructively probe the inner nonvisible structure, pore system, and surface morphology of materials, allowing one to follow microstructure evolution in real-time. A major source of uncertainty in μ-CT is the image resolution that can be attained for a specimen (Elkhoury et al., 2019). Although 2D, other imaging techniques such as thin-section microscopy (Yarmohammadi et al., 2020), scanning electron microscopy (Kuzmin and Skibitskaya, 2017), and atomic force microscopy (Hassenkam et al., 2011) can provide images with higher resolutions and hence combine with μ-CT for better assessment of rock pore space and grains. Several studies have reported a combined use of the MIP and µ-CT for the characterization of material internal structure. Ho and Hutmacher (2006) compared µ-CT with other methods, including the MIP, for the characterization of scaffolds. They argued that the possibility of sample compression and even collapse at very high mercury pressure, the reactions of mercury with some metals such as aluminum, the assumption of the cylindrical pores, and the sample contamination with mercury are major limitations of the MIP. This is while the µ-CT has crucial advantages of being non-destructive and identifying spatial distributions of internal features. They mentioned thresholding and beam hardening as the main concerns of using µ-CT. The applications of MIP and SEM for microstructure evaluation in unsaturated soils were reviewed by Romero and Simms (2008). They pointed out the main limitations of MIP, such as inaccessible pores, including isolated pores and pores which are accessible through smaller ones, non-intruded pores, non-detected pores due to the apparatus minimum pressure limits, and deformation of the sample container. They mentioned that the MIP does not achieve true PSD due to several constraints, including sample treatment and pore accessibility. Fusi and Martinez-Martinez (2013) suggested using µ-CT imaging before and after mercury intrusion to accurately characterize the structural features of low porosity carbonates. The pore systems of complex carbonate formations were studied by Njiekak et al. (2018) using µ-CT, mercury intrusion, and helium porosimetry. They argued that it is crucial to use various techniques when evaluating the multiscale porosity of rocks with a considerable amount of both the connected and non-connected pores. Considering that the µ-CT can provide information on both the connected and non-connected macropores (resolved pores) and the MIP can give connected porosity (resolved and non-resolved pores), combining the data depict an accurate picture of the rock pore system. A complete characterization of a tight sandstone was reported by Cheng et al. (2019) using µ-CT, SEM imaging, and mercury intrusion. The issue of characterizing the evolution of rock structure due to various processes, particularly chemical dissolution using the MIP and/or computed tomography techniques, has been investigated by several researchers. The early work by Swift and Fogler (1977) investigated the evolution of pore size of sandstones due to acidizing using a mercury injection technique. They observed that while the pore sizes of the cores ranged from 0.03 to 30 μm, the dissolution happened only in those pores larger than about 1 μm. Based on these results, they introduced a threshold reactivity diameter concept as a pore size above which the dissolution has the largest effect. Bazin et al. (1995) studied acidizing of limestone cores using the X-ray CT and mercury intrusion. They qualitatively investigated the dissolution pattern by the CT images and evaluated the changes in pore size distribution after acidizing by mercury injection curves. They found that the acidizing process predominantly affected the larger pores. In another study, Burlion et al. (2006) used the μ-CT method to analyze the mortar microstructure variations during an accelerated leaching process. They quantified the evolution of porosity during the process. Their findings are of great importance in the modeling study of concrete durability. A uniform rock structure modification due to dissolution caused by CO2 injection was studied by Egermann et al. (2010). They conducted mercury porosimetry measurements before and after the alteration and found that the distribution of small pore radii was rather unaffected, regardless of the rock types. They observed major modifications of the distributions for the larger pores, but in a different manner according to the particular rock type. Interestingly, it was found that for the low-permeable rock, the dissolution increased the intermediate pore radii, while the large pores of the high-permeable rock mainly increased after the dissolution. This explained the variations in permeability evolution of the rocks where the largest permeability increase was obtained for the rock-type with the most pronounced increase in larger pores (Egermann et al., 2010). Gouze and Luquot (2011) used the μ-CT facility at ESRF (Grenoble, France) to investigate the spatial distribution of the effective hydraulic radius and tortuosity to examine the different porosity–permeability relationships observed for homogeneous and heterogeneous dissolution patterns. They found that for homogeneous dissolution, the permeability increase was due to the decrease of the tortuosity, whereas the combination of tortuosity and hydraulic radius changes were controlling factors for permeability in heterogeneous dissolution. Luquot et al. (2014) investigated the structural changes in limestone samples induced by CO2-rich brine using microtomography. They analyzed various dissolution patterns in relation to the partial pressure of CO2 and found that the initial rock structure strongly controlled the permeability-porosity relationship. In another paper by Luquot et al. (2016), changes in pore size distribution and pore connectivity of a limestone sample induced by an acidic solution were investigated. They compared the evolution of the pore-scale parameters computed by X-ray µ-CT images and laboratory experiments and obtained good agreement between them. Despite the high dependence of image-based computed parameters on the resolution, particularly for the fluid-rock interface, the µ-CT method yielded similar results for the micro- and macro-scale rock properties as the laboratory experiments in a much shorter time and in a non-destructive way. Aside from the changes in pore size distribution the μ-CT method has also been employed to geometrically estimate the changes in specific surface area after chemical dissolution (Noiriel et al., 2004; Noiriel et al., 2009; Gouze and Luquot, 2011). These studies emphasized that the grain and/or grain roughness smaller than the μ-CT resolution could not be measured, and the pore-grain interface area was a priori underestimated. Dal Ferro et al. (2012) conducted a comparative study of the µ-CT and MIP for evaluation of cambisols structure under fertilization treatments. They argued that the MIP measurements were necessary to identify small pores that were not resolved by the µ-CT imaging. They found that compared to the µ-CT method, the MIP-detected micropores were underestimated but macropores overestimated due to the ink-bottle effect. It was also found that only the µ-CT could show the impacts of organic amendments on pore morphology. The microstructure of the original and tectonic coals was explored by the MIP and µ-CT by K. Zhang et al. (2019). In addition to PSD, they used the MIP results to assess the connectivity of the pore system and found that tectonic stress created many noneffective pores with pore-opening constrictions. They also examined the evolution of the morphology of macropores and fractures using µ-CT. This study clearly highlights the significance of employing various techniques when analyzing the microstructure of complex porous media. In a recent interesting paper, Sidiq et al. (2020) reviewed the application of X-ray tomography and MIP for the characterization of concrete pores/cracks content and structure for self-healing cementitious materials. They briefly discuss some sources of limitations, such as the noise level and the thresholding as the major factors contributing to the errors in the μ-CT technique and the applied pressure and the non-intruded internal pores and cracks as the main types of errors in the MIP leading to inaccurate results. Comprehensive reviews regarding the evolution of pore-scale structure and permeability during geochemical processes are provided in Noiriel (2015), Gan et al. (2020) and Khan et al. (2021). Despite previous studies on the study of chemical dissolution in porous materials using various experimental and imaging techniques, there is still a need to accurately address the respective challenges when characterizing the evolution of the internal structure of porous rocks using the two powerful tools of µ-CT and MIP, particularly for complex microporous rocks such as carbonates. In this study, we first aim to explore and compare the scope of application of these two complementary techniques for tracking and characterizing the evolution of the pore structure of a microporous carbonate rock during chemical dissolution. We carefully address the respective challenges and limitations of both methods in characterizing the evolution of rock pore structures subject to reactive flow. In particular, the evolutions of pore size distribution and pore surface area are quantified, and the role of micropores is explored. The second aim is to predict the resulting changes in permeability due to dissolution using the information provided from 3D µ-CT images and MIP tests. In this regard, we consider two well-known permeability estimation models, i.e., the Katz-Thompson and the Kozeny-Carman equations. In particular, we incorporate sub-resolution pores into critical pore diameter to evaluate the degree to which the inclusion of microporosity could enhance the prediction of permeability changes. 2 Experimental methods 2.1 Rock sample The core sample was obtained from the Savonnières-en-Perthois region in eastern France (Flugel, 2010). Fig. 1 shows a photograph of the core sample used in this study. The rock is an oolitic limestone composed of ellipsoidal particles of calcite known as ooids with diameters ranging from a few tens to thousand microns embedded in sparry calcite cement (sparite). Some ooids are preserved, whereas others are completely or partially hollowed out, forming oomoldic pores. Visual observations of the sample revealed the presence of millimeter-sized oomoldic pores. The XRD test showed that the sample is mainly made of calcite. The mineralogical content of the sample is consistent with previous studies by, e.g., Zhang et al. (2018) and De Muynck et al. (2011), in which they mentioned the Savonniere limestone contains 97-99% calcite and 1-3% biotite. 2.2 Experimental procedure In this study, the experiment involved alternating steps of µ-CT imaging and ex-situ core sample alteration by chemical dissolution using 0.25 M ethylenediaminetetraacetic acid (EDTA) solution at pH 12 at ambient conditions. Under these conditions, the dissolution of the sample was mainly performed through ligand-promoted dissolution (Fredd and Fogler, 1998). Here, dissolution at high pH is beneficial in reaching a homogeneous alteration of pore structure within the core sample because the kinetics of the chelation reactions dominates the rate of dissolution. Fig. 2 illustrates the schematic diagram of the setup used for the chemical dissolution experiment. A small core plug with a diameter and length of 7.0 and 20.6 mm, respectively, was drilled out from the original rock specimen and dried in an oven. It was then sealed on the sides using a Fluoropolymer heat shrink and imaged in a dry state using the µ-CT facility at the Australian National University (Sakellariou et al., 2004). The initial porosity of the sample was measured by the saturation method (ϕ 0 = 30.6%), then deionized water was injected at different rates to measure permeability by Darcy's law (k 0 = 58.6 md). The reactive fluid was injected through the sample at rate of 0.36 cm3/min using a motorized syringe pump, and the pressure drop across the sample was monitored by a differential pressure transmitter (ABB Model 2600T, pressure range from 0.0000 to 250.0000 kPa). The reactive injection was terminated after a significant increase in the permeability of the sample (kf = 152.4 md). The final porosity of the core was measured (ϕf = 42.6%) before it was imaged in a dry state again. A 3D registration technique by Latham et al. (2008) was used to superpose the µ-CT image of the disturbed sample to the image of the original unaltered sample. Tomograms in each state were composed of 20483 voxels. Fig. 3 summarizes the lengths and voxel sizes of pre- and post-dissolution scans, as well as the registered images. As schematically shown in Fig. 3, images of the core sample were taken at about 4 mm from the front of the sample to eliminate the inlet effect with a field of view of at least 8.5 mm. The final registered tomograms corresponded to two datasets of 1632 × 1592 × 1664 voxels with a common voxel size of 4.49 μm. 2.3 Scanning electron microscope (SEM) We used SEM to display the finer features of the pore systems than those resolved by the µ-CT imaging. The equipment used for the SEM image acquisition of the studied rock was a Zeiss Ultra-Plus field emission scanning electron microscope (FESEM) located at the Centre for Advanced Microscopy at the Australian National University. The FESEM microscope was operated at a low accelerating voltage (1 kV) and small aperture at a small working distance (around 2 mm) to maximize the resolution. 2.4 Mercury intrusion porosimetry (MIP) tests To analyze the internal pore structure and to compare the results obtained from the μ-CT imaging, we used the mercury intrusion porosimetry (MIP) method as a standard experimental technique to primarily evaluate the pore size distribution (PSD) –or, more strictly, the pore accessibility distribution or pore-throat size distribution– of the specimen. In a mercury intrusion test, the non-wetting mercury invades in a drainage process the connected pore system according to capillary pressure. For each pressure, the change in injected volume is recorded. In a typical MIP test, capillary entry pressure (mercury/air) and the volume of intruded mercury into the pore space of the specimen are directly measured. In this study, MIP testing was performed on the rock samples in both the pre- and post-dissolution conditions using the Micromeritics' AutoPore IV 9500 V1.06 mercury porosimeter with a maximum pressure of 30,000 psia to examine the effect of reactive flow on the micro- and macro-scale properties. We performed three MIP tests in this work. The first test was performed on a sister plug of the intact core sample, marked here as the pre-dissolution sample. The sister plug was chosen very close to the sample imaged. After the chemical dissolution experiment was terminated, the altered sample, marked here as the post-dissolution sample, was split into two pieces, and the MIP tests were performed on both. The first piece, marked as sample piece P1, was located near the injection face (inlet) of the sample where the acquired μ-CT images correspond to this sample piece. The other piece was located far from the injection face of the sample, marked as sample piece P2, and was also used for the MIP study to observe changes in pore size distribution along the length of the core. Referring to Fig. 3, the μ-CT image of the altered sample contains the rock piece P1 being imaged and does not include the rock piece P2. 3 Computational methods 3.1 µ-CT image analysis and image-based simulations 3.1.1 Image processing A number of key limitations exist when applying the µ-CT technique to study intact and successively altered core samples subject to a variety of processes such as dissolution/precipitation and mechanical loading. First, image resolution is one of the most important barriers to capture features at extremely small scales. The resolution can be enhanced by reducing the sample volume scanned. This, in turn, may reduce the size of the image below the representative elementary volume (REV) of the sample, which should be considered for representative macroscale property calculations. Second, the occurrence of image noise is inevitable. Noise degrades the image quality and hence the further image-based characterization accuracy. A number of algorithms have been developed to efficiently reduce noise from tomographic images (Gonzales and Wintz, 1987; Mallat, 1989; Perona and Malik, 1990; Catté et al., 1992). The third difficulty relates to successive imaging of a sample during ex-situ sample alteration. In practice, it is impossible to return the sample to the same position and orientation with micron precision (Wildenschild and Sheppard, 2013). Recently, a procedure has been proposed by Qajar and Arns (2016) in order to accurately compare the successive images of the same core for image-based calculations of micro- and macro-scale property evolution. The proposed approach includes five steps: (1) registration of successive images; (2) rescaling of grey-scale images to a common intensity range; (3) image enhancement using appropriate noise filtering techniques; (4) further rescaling of the filtered images to common intensity range; (5) image segmentation and microporous phases assignment. This approach was previously applied to the images used in this study, accompanied by the statistical uncertainty analysis (Qajar and Arns, 2016). In this paper, a brief description of the methods used to pre-process and segment the image data is given. Key to the success of the analysis given here is (1) registration of the successive images of the core sample subject to chemical dissolution in order to directly observe the evolution of microstructure, including pore structure in particular, and integrate the image data, (2) rescaling of different images of the core sample, taken at different conditions in order to transform the image data to the same reference frame, and (3) segmentation of the registered rescaled successive images of the core sample in order to accurately partition different phases in the images. All other image post-processing steps are based on the segmented images. Regarding the acquired images of the sample in this study, the pre- and post-dissolution registered images were first rescaled to a common 8-bit intensity range in a linear fashion. Then a combination of anisotropic diffusion (Sheppard et al., 2004) and Gaussian smoothing (Wang et al., 2014) filters was applied to reduce noise and enhance the quality of the images. Quantitative measurements of the signal-to-noise ratios of the pore and grain regions were previously reported in (Qajar and Arns, 2016). Finally, the converging active contours (CAC) method (Sheppard et al., 2004) was utilized to partition the images into three phases of macropore (or resolved pore), intermediate, and grain phases. The macropore phase shows pores fully resolved in the µ-CT images, while the intermediate phase voxel contains some mixture of both void and solid within its volume. The pore space in the intermediate phase is commonly referred to as micropores or subresolution pores. The image processing and phase partitioning was accomplished using the Medial Axis and Network Generation (Mango) software tool developed at ANU (Sheppard et al., 2004). The resulting datasets were visualized using an open-source volume rendering software Drishti (Limaye, 2012). 3.1.2 Simulated mercury intrusion and critical pore diameter This paper determined the pore size distributions by simulating the mercury injection on the segmented images. Simulation of mercury intrusion– or capillary drainage transform– represents the invasion-percolation process of the drainage (capillary-dominated flow) experiment (Liu et al., 2017). During this process, the diameter of the largest sphere which can percolate across a connected pore system is called the critical pore diameter (lc ) (Friedman and Seaton, 1998). lc is a unique transport length scale that controls the permeability of porous materials according to the critical path analysis (Scholz et al., 2015). The Euclidean distance transform (EDT) map is needed for the calculation of critical diameters of percolation. The simulation of lc is performed by defining locally for every voxel the diameter of the largest sphere, which fully lies within the pore phase and covers that voxel. At a fixed capillary pressure (equivalent to a pore entry radius), all spheres with radii greater than or equal to the equivalent pore entry radius are considered. We begin with the largest sphere and increment the sphere radius downwards (equivalent to incrementing capillary pressure upwards); the non-wetting phase saturation is measured as the subset of all spheres that have invaded the pore space. One can simulate the complete simulated mercury intrusion curves by considering all possible invading sphere radii (Hilpert and Miller, 2001). We used the Mango software to calculate the simulated mercury curves and the Morphy software (Arns et al., 2001) to calculate the critical pore diameter. 3.1.3 Calculations of specific pore surface area from µ-CT data The specific surface between two phases in a two-phase medium is defined as the area of the interface between the two phases per unit volume of the medium. The inverse of the specific surface is considered as an important characteristic length scale of the medium (Yeong and Torquato, 1998). Various methods have been used to calculate the specific surface of pore space in porous media. In this paper, we compute the evolution of pore surface area from the μ-CT images and the MIP data. High-resolution μ-CT imaging has been used as a reliable tool for quantifying pore surface area and its changes in the course of physical and chemical processes. Concerning the six sides of each voxel, the geometric pore surface area is simply calculated by counting the number of face pixels of pore voxels that are face-adjacent to surrounding solid voxels normalized to the volume of the sample image. Mathematically, (1) S μ − C T = N p x × S p x V i m g where Npx is the number of pore-solid face pixels, Spx is the pixel surface area, Vimg represents image volume, and Sμ-CT is the specific pore surface area. In regard to the μ-CT images of the sample studied in this paper, the pixel surface area (Spx ) is equal to 4.49 × 4.49 (µm)2. The surface area calculations were performed using the MANGO and the Morphy software package. It is worth noting that the most important issue that needs to be addressed is the dependency of the computed surface area on the resolution of the images. This is more challenging when rock samples contain a considerable amount of subresolution porosity. In fact, the μ-CT images cannot provide geometric information at scales below image resolution. Alternatively, the pore surface area can be calculated from the simulated MIP using the same approach adopted for the calculations of pore surface area from the experimental MIP data (see subsection 4.4.3). In this method, the pore surface area calculations are limited to the resolution of the images used for simulations of the mercury intrusion process. 3.1.4 Numerical calculations of permeability and formation factor The permeability is calculated based on a lattice-Boltzmann (LB) simulation of flow in 3D segmented images using the D3Q19 model (Qian and Zhou, 1998). The boundary conditions (BCs) at the solid-fluid interfaces are no-flow BCs that are realized using the bounce-back rule (Martys and Chen, 1996), and the pressure gradient acting on the fluid is simulated by the body force (Ferreol and Rothman, 1995). The formation factor calculations are based on a solution of the Laplace equation with charge conservation boundary conditions (Arns et al., 2001). The 3D image is first converted into a network of resistors by connecting each pair of adjacent voxels with a resistor. Then, a homogeneous electrical field is applied along each of the major tomogram axes, and the potential field is solved for, using a finite element formulation (Bohn and Garboczi, 2003). We assume an idealized case assigning zero conductivity to all mineral phases and a conductivity equal to one to the fluid-filled pore phase. We used the Morphy software for permeability and formation factor calculations. 3.2 Analysis of MIP data 3.2.1 Pore size distribution and pore surface area The MIP data by itself provides no information about the distribution of pore-throat sizes. To generate PSD from MIP data, an appropriate model must be employed. The usual model used to interpret MIP data is the bundle of capillary tubes model (Diamond, 2000). According to this model, the well-known Washburn's equation is applied to estimate the diameter (d) of cylindrical pores intruded by mercury at each pressurizing step (Diamond, 2000): (2) d = − 4 γ cos θ P where γ is the surface tension, θ is the contact angle of mercury on the solid surface, and P is the applied mercury pressure. The typical mercury-limestone surface tension value is 480 dynes/cm, and the contact angle between air (wetting phase) and mercury (non-wetting phase) is around 140° (Howard, 1991; Ho and Hutmacher, 2006). The pore size distribution is then calculated by converting the capillary pressure into the pore-throat size via the Washburn's equation (Eq. (2)) and the cumulative mercury intrusion (in terms of the percentage of pore volume occupied by mercury) into the probability of abundance of each pore in the distribution (Fig. 4 (a), cumulative curve). Another typical graph is the log differential intrusion curve vs. pore-throat size (Fig. 4(a), log differential curve) which assists in the visual identification of the dominant pore modes (Romero and Simms, 2008). It is worth mentioning that while the µ-CT method can detect all macropores (connected, isolated, and dead-end), the MIP can only probe the connected pores and throats where mercury intrudes. The pore-solid specific surface area can be estimated from MIP data. The pore-throat diameter at each incremental pressure step is calculated from the Washburn's equation. Based on a modification made on the assumption of cylindrical pores and the formula for the lateral surface area of a cylinder, it is possible to estimate pore surface area for each pore-throat diameter. Details of equations used for pore surface area calculations from MIP data are presented in Appendix A.1. The cumulative (total) specific pore surface area estimated from MIP measurements is here denoted by SMIP,t . It is also possible to calculate the specific pore surface area from the largest pore diameter down to a certain (or cut-off) diameter. This can help us to define the MIP-based specific pore surface area, denoted by SMIP,mac , based on the cut-off pore diameter equal to the voxel size of the μ-CT image. In other words, SMIP,mac shows the specific surface area of macropores (the pores correspond to the resolved pore space in the μ-CT images). 3.2.2 Pore size statistics Several statistical parameters related to MIP-derived pore-throat size distribution can be defined and quantified. The average pore diameter, Dave , is defined by 4 over the surface-to-pore-volume ratio of the pore space intruded by mercury (see Appendix A.2 for the formula). The median pore-throat diameter (by volume), Dmed,V , is the diameter where 50% of the total mercury volume has been intruded into the open pores of the sample and is correspondent to the 50% of the cumulative intrusion curve. The median pore-throat diameter (by area), Dmed,A , is calculated similar to the median pore-throat diameter by volume but based on pore area distribution. The threshold pore-throat diameter (Dth ) is defined as the largest pore-throat diameter at which significant intruded pore volume is detected (Ma, 2014). In other words, the slope of the cumulative intrusion curve increases abruptly at the threshold diameter. Above the threshold diameter, there is comparatively little mercury intrusion. Aligizaki (2005) stated that the threshold diameter corresponds to the narrowest path in the interconnected pores, as schematically shown in Fig. 4(b). Liu and Winslow (1995) proposed that the threshold pore-throat diameter is a diameter corresponding to the intersection point of the tangent lines drawn on the cumulative distribution curves, as schematically shown in Fig. 4(a). The critical pore-throat diameter (or the characteristic pore diameter, denoted by Dc or lc ) is one of the important pore structure parameters that is closely related to permeability and electrical conductivity. It is the diameter calculated from the Washburn equation for a specific pressure at which mercury first spans the sample and forms an inter-connected pathway throughout the sample, i.e., a continuous filament of mercury extends through the pore network of the sample (Katz and Thompson, 1987; Svendsen et al., 2004). Aligizaki (2005) argued that the critical pore-throat is the most frequently occurring pore size in interconnected pores that allows maximum percolation, as schematically shown in Fig. 4(c). Swanson (1985) and Katz and Thompson (1986) adopted the inflection point of the rapidly rising initial portion of the cumulative intrusion curve to mark that specific pressure. This inflection point corresponds to the steepest slope of the cumulative intrusion curve. i.e., the highest point in the log differential intrusion curve (Gao and Hu, 2013). The determination of Dc is schematically shown in Fig. 4(a). It is believed that the critical pore-throat diameter is the maximum pore-throat diameter that can connect the larger pores and reflect the connectivity of the pore system (Zhu et al., 2019). The pore-throats larger than the critical pore-throat diameter mainly contribute to flow and hence permeability. 4 Results and discussion 4.1 Characteristics of pore space from SEM imaging Fig. 5 (a) shows an SEM micrograph of the whole core cross-section of a 5-mm diameter core sample of the rock with 20 × magnification and a resolution of 4.961 µm. Further investigations were carried out through SEM to probe the µm- and nm-scale pores at higher magnifications. Fig. 5(b-f) illustrates the SEM close-up views of the rock corresponding to locations marked by orange rectangles in Fig. 5(a). In Fig. 5(b-f), individual ooids, ooid center and edge, oomoldic pores, and sparry cement were visualized at extremely high resolution, representing the presence of extremely small pores, e.g., intercrystalline pores within ooid and sparry cement, which normally cannot be resolved in the μ-CT images, but can be detected by the MIP technique. 4.2 Visual inspections of the evolution of pore space from µ-CT imaging Examples of 2D slices through the 3D registered images are shown in Fig. 6 . Preliminary visual inspections of the images clearly show evidence of chemical dissolution of ooids and sparry cement and enlarging of oomoldic pores. Close observations of the dissolution patterns revealed that the chemical dissolution locally enlarged pores across most sample parts with no visible major flow paths (wormholes). In this case, the chemical dissolution resulted in a homogeneous alteration of the sample pore structure. In this paper, the resolved porosity (macroporosity) was separated based on the visual inspections of the images (two-phase segmentation). The three-phase segmentation of the images was then carried out using a compromise between the visual inspections of the images and an approximate match between the image-based total porosity and lab-measured core porosity. An example of slices of the partitioning of the macro and micropore phases in the pre- and post-dissolution core sample is shown in Fig. 7 , where the resolved macropore phase is illustrated in black, the grain phase is given in white, and the micropore phases are shown in greyscale. The intensity of shades of gray is proportional to the void fraction of the microporous phase, where the darker gray color represents the microporous phase with a larger void fraction. The image-based total porosity of the pre-dissolution sample gives the value of 27.8%, of which 12.5% is macroporosity. 50% of the sample image is assigned as an intermediate phase with a microporosity contribution of 15.3%. The segmentation results for the post-dissolution sample show a total porosity of 41.0%, of which 23.5% is macroporosity. In this case, about 40% of the sample image is assigned as an intermediate phase with a microporosity contribution of 17.6%. The segmentation data indicates approximately a double increase in macroporosity but a slight increase in microporosity after the dissolution experiment. A comparison between the experimentally and image-based measured porosity shows that the total image porosity of the post-alteration sample is in a good match with the experimental porosity, while the pre-alteration image porosity was slightly underestimated mainly due to the presence of the higher amount of image noise. Overall, the difference between the experimental and image porosity can be attributed to the experimental measurement error, image quality and noise, segmentation error, and the fact that we may image only part of the sample. It is noted the porosity of the unaltered sample is characterized by 45% of macroporosity. This value was increased to 57.5% after the dissolution representing that the initial microporous-dominant pore structure converted to the pore system dominated by macropores. As a consequence, it was expected that the major effect of the chemical dissolution was to largely increase in macroporosity of the sample. As evidenced later, the image-based results were also confirmed by the performed MIP tests, which showed a significant increase in the number of larger pores after the dissolution experiment. Luquot et al. (2016) found very similar findings in porosity changes, while in another study, the opposite results were obtained (Luquot et al., 2014), in which the authors argued that the enlargement of micropores could induce the local migration of fines. 4.3 Characterization of the evolution of pore size and surface area using the MIP data Before presenting the results of MIP measurements and comparing them with the image-based results, it is essential to address the major limitation of MIP-derived data. The MIP method suffers from its assumptions and hence includes several drawbacks (Diamond, 2000; Gorce and Milestone, 2007; Zhang et al., 2019). First, both the advancing and receding contact angles at pore walls are assumed to be uniform. Second, the MIP method is not able to detect pores of access diameter larger than a certain size due to limited mercury filling pressure at the low-pressure stage of the MIP test. In other words, the MIP data starts at a non-zero saturation according to the entry mercury pressure. An initial increase in mercury pressure is due to mercury entering surface pores and fractures, which may not be detected by the MIP method. Third, it is assumed that the pore structure does not change during MIP measurements. This assumption may not be valid for poorly connected pore structures where the high mercury pressure may break the solid partitions between two adjacent non-connected pores. Cao et al. (2016) argued that MIP data at pressures 70 MPa and above are unreliable and probably not meaningful due to changes in pore structure, including possible particle breakdown and the opening of closed pores. Fourth, the most important limitation is related to the assumptions of the cylindrical pores with uniform radius. In other words, this method neglects the pore irregularities and the fact that the pore system may be composed of large pore bodies interconnected by narrow pores (throats), i.e., ink-bottle pores (Schmitt et al., 2015). Mercury requires a breakthrough pressure to pass through the throats of non-cylindrical pores. Consequently, the distribution curve from the MIP measurement should be interpreted as the pore-throat diameter distribution. Despite these limitations, the MIP method is one of the few techniques that can identify a broad range of pore sizes and quantify porosity, surface area, and other microstructural parameters. 4.3.1 Pore space analysis of intact rock sample Fig. 8 illustrates the MIP cumulative, incremental, and differential intrusion curves of a sister plug of the sample used in the dissolution experiment. The chosen sister plug, marked as the pre-dissolution sample, served as a control sample. The MIP data shows that the rock possessed a wide pore size range, and its pore-throat sizes are distributed over four orders of magnitude, from tens of nanometers to hundred microns. It is worth noting that the MIP test was not able to detect pores of access diameters larger than 180 µm due to limited mercury filling pressure at the low-pressure stage of the test. In other words, the experimental MIP data starts at a non-zero saturation as only pores of diameter 180 µm or less are probed via the MIP apparatus (1 psi entry pressure). This is while the microscopic and µ-CT images of the sample (Figs. 1 and 6) reveal the presence of surface and internal pores of access diameter larger than 180 μm. The SEM micrographs (Fig. 5) and the µ-CT images (Fig. 6) of the rock sample, in combination with the MIP data (Fig. 8) suggest that the pore space of the rock can be classified as macroporosity occurring within ooids (oomoldic porosity) and between ooids and within sparry cement (interoolitic/intergranular macroporosity), as well as microporosity within intact ooids and ooid cortices (intra-oolitic porosity) and sparry cement (interoolitic microporosity). As evident from Fig. 8, the 4.49 μm). From the cumulative intrusion curve, one can conclude that more than half of the sample porosity is only accessible via subresolution pores or micropore space. Even though the incremental intrusion curve in Fig. 8 shows a large variation from the log differential intrusion curve (dV/dlogD pore volume versus pore-throat diameter), the pore size distribution curve of the sample is characterized by a double-peak (bimodal) distribution. Regarding the method proposed by Katz and Thompson (1986) for determining the critical pore-throat diameter and the pore size of the rock sample as a bimodal distribution, we found two critical pore-throat diameters in both the macropore and micropore regions. The first bump in the log differential intrusion curve covers mainly in the pore size range of 10-20 µm, with the pore-throat diameter corresponding to the large pore-mode (the 1st major peak, which occurred at ∼12 µm). This pore-throat size range belongs to the macropore space. The pores with diameters larger than the macro critical pore-throat diameter (∼12 µm) are expected to mostly control the permeability of this rock at the initial condition before the chemical dissolution experiment. The second bump primarily covers the pore sizes between 30 and 400 nm, with the pore-throat diameter corresponding to the small- classification of the pores into macro- and micropores is here based on the resolution of the µ-CT images (i.e., pore mode (the 2nd peak, which occurred at ∼100 nm). This pore-throat size range belongs to the micropore space of the rock. The two bumps in the differential curve suggest that the pore-throat sizes can be divided into three parts: >12 μm, 12-0.1 μm, and <0.1 μm. Table 1 summarizes statistical results on pore sizes obtained from the MIP data for the pre- and post-dissolution states. While the average pore-throat diameter of the intact (pre-dissolution) sample is far below the resolution of the μ-CT image, the median pore-throat diameter by volume approximately coincides with the resolution of the pre-dissolution μ-CT image (4.15 μm in Fig. 3), highlighting the fact that 50% of the porosity of the pre-dissolution sample is accessible via the macropore space region. 4.3.2 of chemical dissolution on pore-throat diameter from MIP data Fig. 9 compares the MIP cumulative, incremental, and log differential intrusion curves of the pre- and post-dissolution samples. The solid, dashed, and dotted lines in Fig. 9 represent the pre-dissolution, post-dissolution sample piece P1, and post-dissolution sample piece P2 MIP curves, respectively. Since significant changes in pre- and post-dissolution MIP curves are observed, we can neglect deviations that could be attributed to the fact that the mercury porosimetry measurements were performed on the sister plug of the sample prior to the dissolution. It is apparent from Fig. 9 that there has been a significant shift in pore-throat size distribution to the right. The number of macropores significantly increased for the pores larger than 20 μm and decreased for smaller ones. A similar observation was obtained for micropores, where the number of larger micropores (between 400 nm and 2 μm) increased while the number of smaller ones decreased. As a result, a large number of original pores, particularly the larger macropores, were enlarged or merged to form larger pores during the dissolution experiment. Since permeability is largely determined by the interconnected macropores with sizes greater than the critical macropore diameter, the conducted chemical dissolution has significantly improved the permeability of the sample by first enlarging the macropores and second by increasing the connectivity of the macropore system. The changes in the connectivity of the pore system of the rock sample were investigated in detail in our previous paper (Qajar and Arns, 2017). Fig. 9 also gives information about the longitudinal variation of the pore-throat size distribution of the post-dissolution sample. A quick look at Fig. 9 and Table 1 points out a small difference between the MIP curves of sample pices P1 and P2, indicating a quasi-uniform dissolution within the sample, and hence confirming the image-based results. A comparison between the MIP data of the post-dissolution sample pieces P1 and P2 indicates a moderately larger increase in the number of larger pores and throats for the near-inlet post-dissolution piece, which confirms the image-based porosity profiles obtained in our previous work (Qajar and Arns, 2016). Table 1 also shows a slightly larger increase in average, median, and critical pore-throat diameters for the near-inlet part of the post-dissolution sample (piece P1). Despite the presence of slightly more dissolution-induced macropores in the near-inlet post-dissolution sample, the dissolution map, as evidenced by the µ-CT images, is very close to uniform along the core length. The changes in the pore-throat size statistics calculated from the MIP data are reported in Table 1. If one estimates the MIP incremental intrusion curves in both pre- and post-dissolution states as unimodal distributions, the calculated mean and median pore diameters suggest that the MIP curves are negative skew, representing the largest occurrence of the pore size is in the macropore region. The mean and median pore diameters significantly increased after the chemical dissolution. It can be argued from the MIP cumulative curves that the threshold pore-throat diameter considerably increased after the dissolution experiment, despite the inability to detect pores of access diameters larger than 180 µm in the MIP tests. Data from Table 1 also indicates that the macro critical pore-throat diameter, corresponding to the top of the first major peak in Fig. 9(c), increased from ∼12 µm to ∼52 µm after the dissolution test. This shows that mercury quickly formed a connected filament through the pore space of the post-dissolution sample during mercury intrusion at the low-pressure stage. As a result, one can conclude that the enlarged oomoldic pores with diameters greater than ∼52 µm govern fluid flow (and hence permeability) in the post-dissolution sample. The micro-critical pore-throat diameter, corresponding to the top of the second major peak in Fig. 9(c), also increased from ∼0.1 µm to ∼1.1 µm. In order to obtain a better understanding of the effect of the chemical dissolution on the evolution of pore-throat size, we consider a relative change in pore-throat number (denoted by R). The corresponding equations used for the calculation of R are given in Appendix A.1. Fig. 10 shows the relative change in the incremental intrusion volume versus pore-throat diameter for both the altered sample pieces and can be interpreted as the ratio of changes in the number of pore-throats for a given pore-throat diameter. The zero R in Fig. 10 means no change in the relative pore-throat number during the chemical dissolution process, whereas the positive and negative R values indicate an increase and decrease in relative pore-throat number, respectively. Fig. 10 clearly shows that the chemical dissolution did not uniformly change the number of different pore-throat diameters. The largest increase is found for pore-throat diameters between 20-100 μm based on the data of altered sample pieces P1 and P2. The most observed increase in the number of pore-throat diameters occurred for pore-throat diameters of 90 and 50 μm based on the MIP data of the altered sample pieces P1 and P2, respectively, with an average value of 70 μm (Fig. 10). Swift and Fogler (1977) defined the threshold reactivity diameter as the pore-throat diameter above which the dissolution has the largest effect. Fig. 10 shows the largest relative increase in pore-throat number occurs at diameters greater than 20 μm. Both MIP tests on pieces P1 and P2 illustrate that the relative change in pore-throat number is close to zero for pore-throat diameters below 20 μm, indicating little or no change in pore-throat number. Consequently, we found that the effectiveness of the chemical dissolution was greatest for larger pores and throats in the macro pore region defined by the μ-CT images. The MIP results are consistent with visual observations of the μ-CT images indicating considerable enlarging of oomoldic and intergranular macropores (Fig. 6). 4.3.3 Analysis of the evolution of specific pore surface area using MIP data We calculated the specific pore area according to the procedure explained in Section 3.2.2. Fig. 11 shows the cumulative specific pore surface area before and after the chemical dissolution experiment. It is evident that pore-throats with diameters below 1 μm contributed to about 95% of the total pore area for both the pre- and post-dissolution samples. The pore-throat diameter is inversely proportional to the specific pore area, which indicates a larger cumulative specific pore area for the post-dissolution sample. However, there was a dramatic decrease when the pore-throat diameter was below 0.08 μm. It can be argued that there are more micro/nanopores and micro/nanothroats found after the chemical dissolution and that these micropores and microthroats contribute to an increase in the pore area but not the pore volume. Table 2 reports the results of the cumulative specific pore surface area from the MIP method for the pre- and post-dissolution samples. It was found that the total specific pore surface area (SMIP,t ) was reduced by 21% after the dissolution (based on the average values of the cumulative specific pore areas of sample pieces P1 and P2). It is also possible to evaluate the changes in pore surface area based on a specific pore-throat diameter. In particular, based on the cut-off pore-throat diameter equal to the voxel size of the μ-CT image, the specific pore surface area (SMIP,mac ) almost remained unchanged after the dissolution based on the information obtained from the sample piece P1. However, SMIP,mac decreased by 8% based on the average MIP data of the sample pieces P1 and P2. 4.4 Image-based quantification of changes in pore size and pore surface area 4.4.1 Analysis of the evolution of pore size from the μ-CT images Various approaches have been adopted to study pore size distribution (PSD) from μ-CT image data. In this paper, we use the distribution of the maximal covering spheres percolating the sample as pore diameter distribution. This can be obtained by simulating mercury intrusion porosimetry (simulated MIP) based on 3D microstructural data provided from the μ-CT image. The simulations were performed on the resolved pore space (macropores) of the whole registered images. Fig. 12 shows a comparison between the simulated MIP curves and the experimental MIP measurements on the core sample. It is worth mentioning that the simulated MIP curves are limited to the voxel size of the images and do not cover the whole pore size range of the sample detected by experimental MIP. From the simulated MIP data, we found that the number of larger macropores (pores larger than ∼40 μm) significantly increased and slightly decreased for smaller ones. A similar analysis of the relative change in macropore number was performed on the simulated MIP data, and it was found that the largest increase in macropore number occurred for a pore diameter of 60 μm, which is in good agreement with the value of 70 μm obtained from the experimental MIP data (Fig. 10). It is noted that the image-based simulated MIP data has a necessary cut-off at porosities associated with pores below the voxel size of the image. Regarding the complex pore structure of the sample, Fig. 12 indicates an acceptable agreement between the experimental and simulated MIP data. As expected, the pore size distribution based on the simulated MIP overestimated the larger pores, which consequently shifted the PSD in the direction of the bigger pores. Table 3 reports the estimated average, threshold, and critical pore diameter from the simulated MIP curves of the pre- and post-dissolution images. Here, the estimations were performed similarly to those obtained from the experimental MIP data. The results show that despite a slight increase in the average pore diameter, the threshold and critical pore diameters increased significantly after the dissolution. In particular, the critical pore diameter increased from 45 to 72 μm. A comparison with the experimental MIP results (Table 1) shows that the image-based method overpredicts the pore size statistics primarily due to the disability of the μ-CT images to detect whole pore space with detailed pore structure. 4.4.2 Effects of chemical dissolution on pore surface area from μ-CT data The μ-CT data can be used to characterize the influence of chemical dissolution on the surface area between pore and solid phases based on the spatially resolved voxel evolution regions. In this technique, the geometric pore surface area is simply calculated by counting the number of direct solid neighbor voxels to each pore voxel. To characterize the evolution of pore surface area, first, the μ-CT method requires a precise 3D registration of the original and altered images in a common coordinate system. Second, the surface area computed from μ-CT images is highly dependent on the resolution of the images. In other words, the existence of a considerable amount of intermediate phase makes the quantification of the pore-solid surface area difficult. The latter problem is more difficult to experimentally deal with, especially when the rock sample contains subresolution porosity. We have conducted a stable chemical dissolution test in a small rock sample to increase the image resolution as much as possible. The μ-CT data is used here to measure the surface area between macropores and the solid phase. The measured surface area involves the surface area of both connected and unconnected pores. The μ-CT data is not able to capture the geometry at scales below the image resolution. Regarding the images of the core sample studied in this paper, the ooid grain roughness, grains smaller than the resolution of the images, and microporosity are not distinguishable in the images. Therefore, we cannot measure the actual –or more strictly, the subresolution or submicron– pore-solid surface area from the images. However, observations of the pre- and post-dissolution images, as well as the mercury porosimetry measurements, show that the conducted chemical dissolution test mainly affects the macropore space. In fact, the surface area of major flow pathways is more important than the surface area in isolated pores or micropores that do not contribute much to fluid flow (Noiriel et al., 2009). Therefore, changes in actual pore-solid surface area are largely influenced by changes in macropore surface area. Consequently, μ-CT can be a useful tool for estimating effective pore surface area changes. Here, the macropore surface area (i.e., the surface area between macropore and non-macropore phases), denoted by Sμ-CT,mac , was estimated by directly counting the number of voxels and faces of the macropore phase on the image as well as by using the simulated MIP curve. In this case, the image-based pore surface area is denoted by Sμ-CT,mac . Fig. 13 shows the variation of specific surface area between macropore and non-macropore phases along the flow direction for each slice of the images. The initial specific surface area of macropores shows local variability, while the macropore specific surface area of the post-dissolution sample exhibits more uniformity, representing the surface area of macropore space changed almost uniformly along the flow direction after the dissolution test. Alternatively, it is possible to evaluate the evolution of macropore surface area using the simulated MIP data. Fig. 14 illustrates the cumulative specific macropore surface area before and after the chemical dissolution experiment. Here, the calculations of the total specific macropore surface area from the simulated MIP data are similar to the procedure adopted for the experimental MIP data analysis for pore surface area. The porosity used in calculating the macropore surface area based on the simulated MIP was the macroporosity, not the total porosity obtained from the segmentation of the images. It is evident from Fig. 14 that the cumulative specific macropore area is larger for the post-dissolution sample. Table 4 summarizes the total macropore-specific surface area values from the image data. 4.5 Discussion: a comparison between the MIP and μ-CT methods in the quantification of the evolution of pore surface area From experimental MIP data (Table 2), we found that the specific total pore surface area decreased by 21%. Even for macropores, the experimental MIP data exhibits an 8% reduction in specific pore surface area. Conversely, the image-based data based on the segmentation into resolved pore space shows a significant increase in pore surface area (Table 4). Since from the experimental MIP data (Fig. 8), more than half the sample pore size is less than the resolution of the images, it is concluded that the evaluation of the pore surface area of the sample based on resolvable pore space from images was inaccurate and ignoring micropores as part of the pore system led to a considerable error. Despite the ability of the MIP technique in the quantification of the pore surface area for a wide range of pore sizes, down to a few nanometers, this method assumes the model of the bundle of capillary tubes, which neglects the actual pore structure of the rocks. On the other hand, the µ-CT method provides a relatively accurate representation of the pore structure. But the weak point in the µ-CT method is that the image-based results are dependent on the image resolution, and ignoring subresolution pores results in a significant error. Although the rock investigated in this study are highly microporous, the dissolution mainly affects the larger pores of the sample in the macropore region. Hence, the use of both methods can give a complementary insight in analysis of the changes in pore surface area. Measurements of the pore surface area with µ-CT reveal that when we ignored microporosity, the specific surface area of macropores increased by an average of ∼ 40% (first column in Table 4). This is primarily due to the enlargement of macropores and is consistent with the fact that the conducted chemical dissolution had the largest effect on the macropores, and macrothroats with diameters greater than 20 µm (Fig. 10). Observations of the µ-CT images (Fig. 6) reveal that the pore size of oomoldic and interoolitic macropores increases, whereas the pore size of oolitic grains (ooids) decreases. In the macropore region of the sample, the dissolution-induced enlargement of oomoldic and interoolitic pores was dominant and increased pore surface area. However, when we look at the experimental MIP data, the total specific pore surface area was reduced by 21% after the dissolution test (first column in Table 2). Even when we calculated the changes in pore surface area from the experimental MIP data for pore sizes down to the cut-off size equal to the resolution of the µ-CT images, we found a slight decrease of 8% in pore surface area (second column in Table 2). 4.6 Predictions of the changes in permeability So far, we have focused on the analysis of the evolution of two characteristic parameters of the pore phase, i.e., pore-size distribution and pore surface area during the chemical dissolution. In this section, we evaluate the evolution of permeability based on the pore-scale information provided by the MIP and μ-CT techniques. This is of great importance since an improved estimation of the evolution of macroscopic rock properties, including porosity and permeability, can potentially assist in managing the risks associated with the reactive processes in sub-surface formations. To evaluate the evolution of permeability during chemical dissolution based on the pore-scale descriptors calculated from the MIP and μ-CT methods, we consider two well-known permeability estimation models, i.e., the Katz-Thomson (KT) and the Kozeny-Carman (KC) equations. The predicted changes in permeability are compared with those calculated directly on the images using the lattice-Boltzmann (LB) flow simulation in 3D segmented images. The KT and KC models are briefly described as follows: Katz-Thompson (KT) equation. One of the most common methods for estimating permeability uses the equation given by Katz and Thompson (1986): (3) k = c K T l c 2 1 F where k is permeability, cKT is a prefactor that depends on the distribution of pore sizes in the rock (Bauget et al., 2005), lc is a characteristic pore size, and F is a dimensionless parameter called the formation factor. The constant cKT is of the order of 1/226 (Katz and Thompson, 1986). The characteristic pore size in Eq. (3) is usually taken to be the critical pore diameter, which was directly measured here from the MIP curves (see the two last columns in Table 1). We calculated the critical pore diameter and formation factors for several subvolumes of the images as previously mentioned in Sec. 3.1.5. The calculations were based on the resolved pore space. Alternatively, the formation factor can be estimated from the experimental MIP data. Eq. (4) was proposed by Katz and Thompson (1987) to estimate the formation factor from the same MIP data used to measure the characteristic pore size, lc : (4) 1 F = l max e l c ϕ S ( l max e ) where ϕ is the porosity, l m a x e is a pore size at which hydraulic conductance is maximum, and S ( l m a x e ) is the fraction of total porosity filled at l m a x e , or in other words the fractional volume of connected pore space involving pores with sizes l m a x e and larger. Katz and Thompson (1987) argued that l m a x e can be estimated from the following relation for very broad size ditributions: (5) l max e = 0.34 l c According to the KT equation, a non-wetting fluid can have access from one face of the sample to the other when the driving pressure is sufficient to allow the fluid to penetrate the smallest pore-throat of diameter lc in the most efficient conducting pathway (Cerepi et al., 2003). Kozeny-Carman (KC) equation. One of the widely used models for permeability is the Kozeny-Carman equation (Torquato, 2002): (6) k = c K C ϕ 3 S P 2 where SP is the specific pore surface area (per unit bulk volume) and cKC , called the Kozeny's constant, is a constant and generally depends on pore geometry and tortuosity. The specific pore surface area was calculated using both the MIP (see Table 2) and the μ-CT images (see Table 4). As previously observed, the chemical dissolution locally enlarged pores across most parts of the sample with no visible major flow paths (wormholes) and led to a quasi-uniform dissolution pattern within the sample. At this condition, it is reasonable to assume that the constants of the KT and KC equations (i.e., cKT and cKC ) remained unchanged during the uniform dissolution pattern. Hence, we propose to use the following relations to assess changes in the permeability of rock samples due to chemical dissolution based on the KT equation using the critical pore size and formation factor data and based on the KC equation using the porosity and specific surface area data: (7) ( k k 0 ) K T − μ C T = ( l c l c 0 ) 2 ( F 0 F ) (8) ( k k 0 ) K C = ( ϕ ϕ 0 ) 3 ( S P 0 S P ) 2 where the subscript 0 refers to the properties before the dissolution test. Inserting Eqs. (4) and (5) in Eq. (7) leads to: (9) ( k k 0 ) K T − M I P = ( l c l c 0 ) 2 ( ϕ ϕ 0 ) ( S ( 0.34 l c ) S ( 0.34 l c 0 ) ) Regarding the post-dissolution MIP data, we averaged between the results of the sample pieces P1 and P2 for the calculations of changes in permeability in order to provide a more accurate comparison between the MIP-based results and the lab-based measurements. It is worth mentioning that if the purpose is to compare the image-based and MIP-based results, we should use the MIP data of the sample piece P1, because the μ-CT image of the post-dissolution sample corresponded to the sample piece P1. 4.6.1 The evolution of porosity Table 5 compares the experimentally measured porosity values in pre- and post-alteration states and the porosity ratios (ϕ/ϕ 0, i.e., the ratio of post- to pre-dissolution porosity) and the corresponding results obtained by the MIP and µ-CT methods. The µCT-based porosity results were obtained for the image volumes depicted in Fig. 3 using the CAC segmentation method. It is worth mentioning that the macroporosity was separated based on the visual inspections of the µ-CT images in the first segmentation stage, known as two-phase segmentation. In the second stage of phase partitioning (i.e., three-phase segmentation), the phase separations were carried out using a compromise between the visual inspections of the images and an approximate match between the image-based total porosity and lab-measured core porosity. The total porosity of the sample derived from the μ-CT images is obviously in a reasonable match with the values measured experimentally in the lab by the saturation method. The differences, particularly for the pre-dissolution sample, can be attributed to the image noise and the fact that we imaged only part of the sample, which indicates that the μ-CT calculated parameters are restricted to the part of the sample imaged. Regarding the MIP method, the total porosity at the pre-dissolution condition is somewhat lower than the lab-measured value, mainly because the MIP measurements were performed on the sister plug of the sample, not the sample itself, before the dissolution. Measurements of the total porosity of the post-dissolution sample using the MIP method exhibit an excellent agreement with respect to the lab-measured experimental data. For both the MIP and μ-CT methods, it is possible to calculate the macroporosity ratio, (ϕ/ϕ 0)macro, the microporosity ratio, (ϕ/ϕ 0)micro, and the total porosity ratio, (ϕ/ϕ 0)total, while the formers (i.e., the macro- and microporosity ratio) are not possible to be measured in the lab. Regarding the resolved pore space (macropore region), both the MIP and μ-CT methods revealed that the macroporosity has approximately doubled. In contrast, the microporosity slightly increased after the dissolution, as indicated by both the MIP and μ-CT data. 4.6.2 Evaluations of the changes in permeability based on the macropore space The numerical calculations of the critical pore diameter, formation factor, and the LBM permeability were performed on several subvolumes with the full section-area of the image data and 200 or 400-pixel thicknesses in along the flow axis. Multiple measurements on image data give computational predictions of properties rather than obtaining a single data point. We have previously found that the size of subvolumes was sufficiently larger than the pore-size correlation length, which allows producing representative results. Fig. 15 shows the ratio of altered (post-dissolution) to unaltered (pre-dissolution) permeability (k/k 0) calculated directly by the LBM method plotted against the estimated (k/k 0) by the KT and KC models using the image-based data (based on the resolved pore space or macropores). We found that the estimated permeability ratios from the KT model (Eq. (7)) were in the same order of magnitude as those obtained with the LBM. On the contrary, the estimated permeability ratios from the KC model (Eq. (8)) differed from the LBM results between one and three orders of magnitude, highlighting that ignoring the micropore surface area significantly underestimated the permeability ratio. This result demonstrates that the KT model can reasonably predict the permeability evolution of the studied complex carbonate sample from the image-based critical pore diameter and formation factor data of the resolved pore space. Furthermore, the result suggests that the critical pore diameter (lc ) of the macroporous phase governs the fluid flow through the microporous carbonate sample, and hence it is an appropriate length scale for the prediction of the dissolution-induced permeability evolution of the studied rock sample during the uniform dissolution processes. It is also of interest here to compare the average values of the permeability ratios (k/k 0) obtained from the KT model and LBM method for the images subvolumes. In our case where the subvolumes are arranged in series and the permeability changes only perpendicular to the flow direction, the harmonic average provides the effective permeability. Assuming constant cKT, the average values of the permeability ratios (k/k 0) are given by: (10) ( k ¯ k ¯ 0 ) K T − μ C T = ∑ i = 1 n ( F 0 , i l c 0 , i 2 ) ∑ i = 1 n ( F i l c , i 2 ) Table 6 compares the average post-dissolution permeability ratio to that of pre-dissolution calculated by the LBM and the corresponding values estimated by the KT model using the MIP or µ-CT data. The reported data in the first row of Table 6, i.e., (k/k 0)macro, were based on the resolved macroporous phase. A comparison with the LBM permeability ratio shows that the KT model reasonably predicts the permeability ratio using the data of the critical pore diameter and the formation factor. The difference in results between the LBM and the KT models based on the MIP data can be attributed to the inability to detect pores of access diameters larger than 180 µm due to the limited mercury filling pressure at the low-pressure stage of the MIP test. This led to difficulty estimating reliable critical pore-throat diameter, particularly for the post-dissolution MIP cumulative curves (wee Fig. 9). The image-based method also significantly overpredicted the critical pore size due primarily to resolution limitations. Note that the sample studied in this paper is highly microporous in both the pre- and post-dissolution states, as evidenced by the MIP curves (Fig. 9). So, it is anticipated that ignoring micropore space may induce considerable uncertainty in predicting the changes in permeability. 4.6.3 Incorporation of microporosity into predictions of changes in permeability The presence of subresolution pore space or microporosity and its potential role in flow and transport is of great importance and has been investigated in a few studies (Knackstedt et al., 2006; Apourvari and Arns, 2014; Soulaine et al., 2016). Even though we found that the macropore system mainly controlled the permeability of the rock sample and its changes, it is intriguing to explore the effects of micropores on the evolution of permeability during the chemical dissolution experiment. This is important here because the rock sample considered in this study contained a high amount of microporosity. To assess the effect of microporosity on the performance of the KT method as a prediction model, the LBM permeability and the critical pore diameter were calculated for a pore phase that comprised of macropores and the largest quartile of micropores. In this regard, the macropore voxels were merged with the microporous voxels with a porosity of 100% down to a porosity of 75.5%, and the calculations (along the flow axis) were performed for this combined phase. The results are reported in the second row of Table 6, i.e., (k/k 0)macro+micro. It is revealed that the inclusion of microporosity into property calculations significantly reduced the permeability ratio values and generally improved the predictions of changes in permeability. This suggests that the evolution of micropores during the chemical dissolution experiment affected the changes in permeability to some extent. It is concluded that the two-phase segmentation of the images with neglecting the spatial distribution of micropores increased the uncertainty in predicting the changes in permeability using the KT method as a predictive model based on the image data. 5 Summary and conclusions The research presented and discussed in this paper explores the use of mercury intrusion and 3D pore-scale imaging via µ-CT to characterize the influence of chemical dissolution on the pore structure of a carbonate sample. We have addressed several challenges behind implementing both approaches for reactive flow study in microporous porous media, limiting the usability of the acquired data. In particular, chemical dissolution experiments should be conducted on smaller core samples (typically at a millimeter-scale) to acquire higher resolution images. This leads to the challenge of characterizing the microstructure of the cores at a higher resolution and then connecting it to the macro-scale descriptors and properties. We obtained experimental control points in the form of a quasi-uniform dissolution pattern within a millimeter-scale core sample which allowed us to connect the pore-scale structural evolution to changes in macroscopic properties. Both the μ-CT images and MIP revealed that the dissolution significantly enlarged the pores of the sample in a quasi-uniform manner with no visible/detected wormholes. Regarding the macropore space, the MIP method cannot detect pores of access diameters larger than a certain size due to limited mercury filling pressure at the low-pressure stage of the test. This is while the evolution of macropores can be accurately quantified from the µ-CT images. The MIP results showed that the total pore surface area was reduced after the dissolution, while the μ-CT method showed an increase in the surface area of resolved macropores. While the MIP method can probe the tiny pores down to a few tens of nanometers and hence a more realistic estimation of parameters such as surface area of pores, the µ-CT, despite being limited in resolution, provides a spatial description of the dissolution pattern and can be used to directly study the percolation properties and simulate mercury intrusion through the pore space. Furthermore, the µ-CT enables us to provide multiple measurements and hence can be used for predictive models. This is while the MIP and other usual laboratory techniques provide single measurements and are not able to determine the spatial distribution of micro- and macro-scale properties within a core sample. We evaluated the fractional changes in permeability using the Katz-Thompson (KT) and Kozeny-Carman (KC) relations based on the calculated critical pore diameter, pore surface area, and formation factor. Compared to the LBM method, we found that the KT could predict the ratio of altered (post-dissolution) to unaltered (pre-dissolution) permeability more accurately. This suggests that the evolution of permeability in the core sample is controlled by the changes in the critical pore diameter. Based on the image data, the KC model failed to predict the permeability ratio due mainly to ignoring the micropores in pore surface area calculations. It was also found that the inclusion of microporosity in the critical pore diameter calculations improved the predictions of changes in permeability. The results of this paper indicated that although large macropores were mainly affected by the chemical dissolution, the evolution of micropores exhibited a moderate effect when predicting changes in the permeability of the rock sample. CRediT authorship contribution statement Jafar Qajar: Conceptualization, Formal analysis, Investigation, Methodology, Software, Validation, Writing – original draft. Christoph H. Arns: Supervision, Project administration, Funding acquisition, Writing – review & editing. Declaration of Competing Interest The authors declare the following financial interests/personal relationships which may be considered as potential competing interests: Christoph H. Arns reports financial support was provided by Australian Research Council. Christoph H. Arns reports a relationship with Australian Research Council that includes: funding grants. Acknowledgement CHA acknowledges the Australian Research Council for an Australian Research Fellowship (DP0881112). The authors thank the National Computing Infrastructure (NCI) of Australia for computing support through the national merit allocation scheme. Appendix A.1. Calculations of MIP-derived parameters A.1.1. MIP-derived Pore surface area From MIP data, the pore-throat diameter at each incremental pressure step is calculated from the Washburn's equation (Eq. (2)). To modify the assumption of cylindrical pores with uniform radii, a mean diameter for the ith injection step is given by: (A.1) D m i = D i + D i − 1 2 The cumulative specific intrusion volume for the ith step is calculated by: (A.2) I i = V i V s a m p l e where Vi is the intrusion mercury volume at the ith step and Vsample is the sample volume tested by the MIP technique. From Ii , one can compute the incremental specific intrusion volume for the ith step, Ji : (A.3) J i = I i − I i − 1 The incremental specific pore area for the ith step is given by: (A.4) A i = 4 × J i D m i The cumulative (total) specific pore surface area estimated from MIP measurements (SMIP,t ) is then calculated from summation of incremental specific pore areas: (A.5) S M I C P , t = A i t = ∑ i A i A.1.2. Average pore diameter The MIP-based average pore diameter is defined by: (A.6) D a v e = 4 I i t A i t where Iit and Ait are the total specific intrusion volume (i.e., the porosity) and the total specific pore surface area, respectively. In other words, Dave is 4 over the surface-to-pore-volume ratio of the pore space intruded by mercury (i.e., Ait /Iit ). A.1.3. Relative change in pore-throat number A relative change in pore-throat number, R, from MIP data is calculated by: (A.7) R = J i , p o s t * − J i , p r e * J i , p r e * where J i * is the percentage of the intrusion mercury volume at the ith step and the subscripts “pre” and “post” refer to the pre- and post-dissolution states, respectively. J i * is defined by: (A.8) J i * = J i J i t × 100 where Ji is the incremental specific intrusion volume for the ith step defined by Eq. (A.3) and Jit is the total incremental specific intrusion volume." )

[['classification',
  'ers the pore sizes between 30 and 400 nm, with the pore-throat diameter corresponding to the small- classification of the pores into macro- and micropores is here based on the resolution of the µ-CT images (i.e., p'],
 ['prediction',
  'ges in the permeability of the rock using the Katz-Thompson (KT) and Kozeny-Carman (KC) models. The predictions are based on two characteristic parameters of the pore phase, i.e., the critical pore diameter and'],
 ['prediction',
  'ical pore diameter to evaluate the degree to which the inclusion of microporosity could enhance the prediction of permeability changes. 2 Experimental methods 2.1 Rock sample The core sample was obtained from t'],
 ['prediction',
  '00-pixel thicknesses in along the flow axis. Multiple measurements on image data give computational predictions of properties rather than obtaining a single data point. We have previously found that the size of'],
 ['prediction',
  ' flow through the microporous carbonate s

In [27]:
keyi = 0

In [9]:
#get the full text
def fullText(i, keyi):
    doi = papers.iat[i,4]
#     done = False
#     print(doi)
    try:
        js = requests.get(
            f"https://api.elsevier.com/content/article/doi/{doi}",
            headers = {"X-ELS-APIKey":keys[keyi], "Accept":"application/json"}
,
        )
        
        r = js.json()

        full_text = r['full-text-retrieval-response']['originalText']
#         full_text = re.sub(r'^https?:\/\/.[^\s]+', '', full_text)
#         print(full_text)
        
        abstr = r['full-text-retrieval-response']['coredata']['dc:description'][:25]
        
        index = full_text.find(abstr)
        if(index==-1):
            other.append(i)
            return None
        full_text = full_text[index:]
        
        full_text = ''.join(full_text.split('References')[:-1])

        full_text = full_text.lower().replace('/', ' ').replace('-', ' ').replace('\'', ' ').replace('_', ' ').replace('.', ' ').replace('(', '').replace(')', '').replace('*', '').replace(',', '')
    
#         print(full_text)
        keyi+=1
        
        return full_text
    except Exception as e:
        if js.status_code == 404:
            no_data.append(i)
        elif js.status_code ==429:
            print(str(e))
            keyi+=1
            return fullText(i, (keyi)%9)
        

        else:
            error_df.loc[len(error_df.index)] = [i,doi]
        return None
    


In [29]:
buzzwords_found = pd.DataFrame(columns = ["index", "doi", "buzz_id","category", "subword"])
contexts = pd.DataFrame(columns = ["index", "doi", "context"])
error_df = pd.DataFrame(columns = ["index", "doi"])

In [31]:
for i in range(papers.shape[0]):
    ft = fullText(i, keyi)
    
    if ft == None:
        continue
#     print(ft)
    for j in range(words.shape[0]):
        subterms = words.iat[j, 1]
#         print(subterms)
        for found in find_keywords(subterms, ft):
            contexts.append(found[1])
            buzzwords_found.loc[len(word_contexts.index)] = [i, words.iat[j,0], found[0]]

17512
17513
17514
17515
17516
17517
17518
17519
17520
17521
17522
17523
17524
17525
17526
17527
17528
17529
17530
17531
17532
17533
17534
17535
17536
17537
17538
17539
17540
17541
17542
17543
17544
17545
17546
17547
17548
17549
17550
17551
17552
17553
17554
17555
17556
17557
17558
17559
17560
17561
17562
17563
17564
17565
17566
17567
17568
17569
17570
17571
17572
17573
17574
17575
17576
17577
17578
17579
17580
17581
17582
17583
17584
17585
17586
17587
17588
17589
17590
17591
17592
17593
17594
17595
17596
17597
17598
17599
17600
17601
17602
17603
17604
17605
17606
17607
17608
17609
17610
17611
17612
17613
17614
17615
17616
17617
17618
17619
17620
17621
17622
17623
17624
17625
17626
17627
17628
17629
17630
17631
17632
17633
17634
17635
17636
17637
17638
17639
17640
17641
17642
17643
17644
17645
17646
17647
17648
17649
17650
17651
17652
17653
17654
17655
17656
17657
17658
17659
17660
17661
17662
17663
17664
17665
17666
17667
17668
17669
17670
17671
17672
17673
17674
17675
17676
17677
1767

18878
18879
18880
18881
18882
18883
18884
18885
18886
18887
18888
18889
18890
18891
18892
18893
18894
18895
18896
18897
18898
18899
18900
18901
18902
18903
18904
18905
18906
18907
18908
18909
18910
18911
18912
18913
18914
18915
18916
18917
18918
18919
18920
18921
18922
18923
18924
18925
18926
18927
18928
18929
18930
18931
18932
18933
18934
18935
18936
18937
18938
18939
18940
18941
18942
18943
18944
18945
18946
18947
18948
18949
18950
18951
18952
18953
18954
18955
18956
18957
18958
18959
18960
18961
18962
18963
18964
18965
18966
18967
18968
18969
18970
18971
18972
18973
18974
18975
18976
18977
18978
18979
18980
18981
18982
18983
18984
18985
18986
18987
18988
18989
18990
18991
18992
18993
18994
18995
18996
18997
18998
18999
19000
19001
19002
19003
19004
19005
19006
19007
19008
19009
19010
19011
19012
19013
19014
19015
19016
19017
19018
19019
19020
19021
19022
19023
19024
19025
19026
19027
19028
19029
19030
19031
19032
19033
19034
19035
19036
19037
19038
19039
19040
19041
19042
19043
1904

20244
20245
20246
20247
20248
20249
20250
20251
20252
20253
20254
20255
20256
20257
20258
20259
20260
20261
20262
20263
20264
20265
20266
20267
20268
20269
20270
20271
20272
20273
20274
20275
20276
20277
20278
20279
20280
20281
20282
20283
20284
20285
20286
20287
20288
20289
20290
20291
20292
20293
20294
20295
20296
20297
20298
20299
20300
20301
20302
20303
20304
20305
20306
20307
20308
20309
20310
20311
20312
20313
20314
20315
20316
20317
20318
20319
20320
20321
20322
20323
20324
20325
20326
20327
20328
20329
20330
20331
20332
20333
20334
20335
20336
20337
20338
20339
20340
20341
20342
20343
20344
20345
20346
20347
20348
20349
20350
20351
20352
20353
20354
20355
20356
20357
20358
20359
20360
20361
20362
20363
20364
20365
20366
20367
20368
20369
20370
20371
20372
20373
20374
20375
20376
20377
20378
20379
20380
20381
20382
20383
20384
20385
20386
20387
20388
20389
20390
20391
20392
20393
20394
20395
20396
20397
20398
20399
20400
20401
20402
20403
20404
20405
20406
20407
20408
20409
2041

21610
21611
21612
21613
21614
21615
21616
21617
21618
21619
21620
21621
21622
21623
21624
21625
21626
21627
21628
21629
21630
21631
21632
21633
21634
21635
21636
21637
21638
21639
21640
21641
21642
21643
21644
21645
21646
21647
21648
21649
21650
21651
21652
21653
21654
21655
21656
21657
21658
21659
21660
21661
21662
21663
21664
21665
21666
21667
21668
21669
21670
21671
21672
21673
21674
21675
21676
21677
21678
21679
21680
21681
21682
21683
21684
21685
21686
21687
21688
21689
21690
21691
21692
21693
21694
21695
21696
21697
21698
21699
21700
21701
21702
21703
21704
21705
21706
21707
21708
21709
21710
21711
21712
21713
21714
21715
21716
21717
21718
21719
21720
21721
21722
21723
21724
21725
21726
21727
21728
21729
21730
21731
21732
21733
21734
21735
21736
21737
21738
21739
21740
21741
21742
21743
21744
21745
21746
21747
21748
21749
21750
21751
21752
21753
21754
21755
21756
21757
21758
21759
21760
21761
21762
21763
21764
21765
21766
21767
21768
21769
21770
21771
21772
21773
21774
21775
2177

22976
22977
22978
22979
22980
22981
22982
22983
22984
22985
22986
22987
22988
22989
22990
22991
22992
22993
22994
22995
22996
22997
22998
22999
23000
23001
23002
23003
23004
23005
23006
23007
23008
23009
23010
23011
23012
23013
23014
23015
23016
23017
23018
23019
23020
23021
23022
23023
23024
23025
23026
23027
23028
23029
23030
23031
23032
23033
23034
23035
23036
23037
23038
23039
23040
23041
23042
23043
23044
23045
23046
23047
23048
23049
23050
23051
23052
23053
23054
23055
23056
23057
23058
23059
23060
23061
23062
23063
23064
23065
23066
23067
23068
23069
23070
23071
23072
23073
23074
23075
23076
23077
23078
23079
23080
23081
23082
23083
23084
23085
23086
23087
23088
23089
23090
23091
23092
23093
23094
23095
23096
23097
23098
23099
23100
23101
23102
23103
23104
23105
23106
23107
23108
23109
23110
23111
23112
23113
23114
23115
23116
23117
23118
23119
23120
23121
23122
23123
23124
23125
23126
23127
23128
23129
23130
23131
23132
23133
23134
23135
23136
23137
23138
23139
23140
23141
2314

24342
24343
24344
24345
24346
24347
24348
24349
24350
24351
24352
24353
24354
24355
24356
24357
24358
24359
24360
24361
24362
24363
24364
24365
24366
24367
24368
24369
24370
24371
24372
24373
24374
24375
24376
24377
24378
24379
24380
24381
24382
24383
24384
24385
24386
24387
24388
24389
24390
24391
24392
24393
24394
24395
24396
24397
24398
24399
24400
24401
24402
24403
24404
24405
24406
24407
24408
24409
24410
24411
24412
24413
24414
24415
24416
24417
24418
24419
24420
24421
24422
24423
24424
24425
24426
24427
24428
24429
24430
24431
24432
24433
24434
24435
24436
24437
24438
24439
24440
24441
24442
24443
24444
24445
24446
24447
24448
24449
24450
24451
24452
24453
24454
24455
24456
24457
24458
24459
24460
24461
24462
24463
24464
24465
24466
24467
24468
24469
24470
24471
24472
24473
24474
24475
24476
24477
24478
24479
24480
24481
24482
24483
24484
24485
24486
24487
24488
24489
24490
24491
24492
24493
24494
24495
24496
24497
24498
24499
24500
24501
24502
24503
24504
24505
24506
24507
2450

25708
25709
25710
25711
25712
25713
25714
25715
25716
25717
25718
25719
25720
25721
25722
25723
25724
25725
25726
25727
25728
25729
25730
25731
25732
25733
25734
25735
25736
25737
25738
25739
25740
25741
25742
25743
25744
25745
25746
25747
25748
25749
25750
25751
25752
25753
25754
25755
25756
25757
25758
25759
25760
25761
25762
25763
25764
25765
25766
25767
25768
25769
25770
25771
25772
25773
25774
25775
25776
25777
25778
25779
25780
25781
25782
25783
25784
25785
25786
25787
25788
25789
25790
25791
25792
25793
25794
25795
25796
25797
25798
25799
25800
25801
25802
25803
25804
25805
25806
25807
25808
25809
25810
25811
25812
25813
25814
25815
25816
25817
25818
25819
25820
25821
25822
25823
25824
25825
25826
25827
25828
25829
25830
25831
25832
25833
25834
25835
25836
25837
25838
25839
25840
25841
25842
25843
25844
25845
25846
25847
25848
25849
25850
25851
25852
25853
25854
25855
25856
25857
25858
25859
25860
25861
25862
25863
25864
25865
25866
25867
25868
25869
25870
25871
25872
25873
2587

27074
27075
27076
27077
27078
27079
27080
27081
27082
27083
27084
27085
27086
27087
27088
27089
27090
27091
27092
27093
27094
27095
27096
27097
27098
27099
27100
27101
27102
27103
27104
27105
27106
27107
27108
27109
27110
27111
27112
27113
27114
27115
27116
27117
27118
27119
27120
27121
27122
27123
27124
27125
27126
27127
27128
27129
27130
27131
27132
27133
27134
27135
27136
27137
27138
27139
27140
27141
27142
27143
27144
27145
27146
27147
27148
27149
27150
27151
27152
27153
27154
27155
27156
27157
27158
27159
27160
27161
27162
27163
27164
27165
27166
27167
27168
27169
27170
27171
27172
27173
27174
27175
27176
27177
27178
27179
27180
27181
27182
27183
27184
27185
27186
27187
27188
27189
27190
27191
27192
27193
27194
27195
27196
27197
27198
27199
27200
27201
27202
27203
27204
27205
27206
27207
27208
27209
27210
27211
27212
27213
27214
27215
27216
27217
27218
27219
27220
27221
27222
27223
27224
27225
27226
27227
27228
27229
27230
27231
27232
27233
27234
27235
27236
27237
27238
27239
2724

In [33]:
word_contexts.to_csv(os.getcwd().replace('scripts', 'data')+'/word_contexts.csv', index = False)




In [47]:
no_data

[]

In [39]:

pd.DataFrame(columns = ['index'], data = other).to_csv(os.getcwd().replace('scripts', 'data')+'/full_text_other_error.csv', index = False)



In [29]:
word_contexts = word_contexts.loc[:696970].drop('context', axis = 1)

In [32]:
word_contexts

paper_id                            category   sub_word
0               0                            Modeling   modeling
1               0                            Modeling   modeling
2               0                            Modeling   modeling
3               0                            Modeling   modeling
4               0                            Modeling   modeling
...           ...                                 ...        ...
1111686     27399  Infrastructure as a Service (IaaS)  databases
1111687     27399  Infrastructure as a Service (IaaS)  databases
1111688     27399  Infrastructure as a Service (IaaS)  databases
1111689     27399  Infrastructure as a Service (IaaS)  databases
1111690     27399  Infrastructure as a Service (IaaS)  databases

[1111691 rows x 3 columns]

In [23]:
contexts = word_contexts['context'].tolist()

In [35]:
pd.DataFrame(columns = ['context'], data = contexts).to_csv(os.getcwd().replace('scripts', 'data')+'/paper_contexts.csv', index = False)

